# Quantum Tomography Overview

* **Author:** Gadi Aleksandrowicz (gadia@il.ibm.com)
* **Last Updated:** January 24, 2019



In [1]:
import numpy as np

## The General Theory

The goal of quantum tomography is obtain data on quantum systems by performing several different measurements. In Qiskit Ingis we are currently concerened with the following two tomography tasks:

1. **Quantum state tomograhpy**: Given a quantum state described by a density operator $\rho$, obtain a matrix representation for $\rho$.
2. **Quantum process tomograhpy**: Given a quantum channel $\mathcal{E}$, obtain a matrix representation (the **Choi matrix**) for $\mathcal{E}$.

In both cases we rely on the assumption that we have access to a large number of identical copies of the system and so can peform several different measures on it.

We can roughly split the tomography process to three stages:
1. Preperation: Add suitable initialization/measurement devices to the quantum system.
2. Experiment: Obtain measurement data from the quantum system.
3. Tomography: Use the obtained data to reconstruct the system's description.

Steps 1 and 2 are related to the quantum system being studied, whereas step 3 is a classical computation which can be carried out on standard computers.

## State Tomography Overview

Let $\rho$ be the density operator of a quantum state. If $\rho$ can be reliably reproduced, it can be subjected to several measurements with respect to different operators; this data can be used to reconstruct $\rho$ or a close approximation for it in several different methods.

### Example: 1-qubit reconstruction using the Pauli basis

Given the Pauli matrices $
I=\left(\begin{array}{cc}
1 & 0\\
0 & 1
\end{array}\right),
X=\left(\begin{array}{cc}
0 & 1\\
1 & 0
\end{array}\right),
Y=\left(\begin{array}{cc}
0 & -i\\
i & 0
\end{array}\right),
Z=\left(\begin{array}{cc}
1 & 0\\
0 & -1
\end{array}\right)$


In [2]:
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

It is easy to see they constitute an orthonormal basis for $M_2(\mathbb{C})$ with respect to the Hilbert-Schmidt inner product $\left\langle A,B\right\rangle =\frac{1}{2}\text{tr}\left(B^{\dagger}A\right)$

In [3]:
def HS_product(A,B):
    return np.trace(np.conj(B).T @ A)


And hence,

$$ \rho =\left\langle \rho,I\right\rangle I+\left\langle \rho,X\right\rangle X+\left\langle \rho,Y\right\rangle Y+\left\langle \rho,Z\right\rangle Z = $$

$$=\frac{\text{tr}\left(\rho\right)I+\text{tr}\left(X\rho\right)X+\text{tr}\left(Y\rho\right)Y+\text{tr}\left(Z\rho\right)Z}{2}$$

The values of $\text{tr}\left(X\rho\right), \text{tr}\left(Y\rho\right), \text{tr}\left(Z\rho\right)$ can be approximated by repeated measuring in the $X, Y$ and $Z$ bases. Since $\text{tr}\left(\rho\right)=1$ there is no need for additional measurements for the coefficient of $I$.

### Example: 1-qubit Linear inversion

The above method can be rephrased in more general form. First, any hermitian operator $H$ has a spectral decomposition of the form $H=\sum \lambda_i P_i$ where $\lambda_i$ is an eigenvalue of $H$ and $P_i$ is the projection operator to the corresponding eigenspace. For the hermitian operators $X,Y,Z$ whose eigenvalues are 1 and -1 we can therefore write

* $X = P^X_0-P^X_1$
* $Y = P^Y_0-P^Y_1$
* $Z = P^Z_0-P^Z_1$

Where



$$P^X_0=\frac{1}{2}\left(\begin{array}{cc}
1 & 1\\
1 & 1
\end{array}\right), P^X_1=\frac{1}{2}\left(\begin{array}{cc}
1 & -1\\
-1 & 1
\end{array}\right)$$

$$P^Y_0=\frac{1}{2}\left(\begin{array}{cc}
1 & -i\\
i & 1
\end{array}\right), P^Y_1=\frac{1}{2}\left(\begin{array}{cc}
1 & i\\
-i & 1
\end{array}\right)$$

$$P^Z_0=\left(\begin{array}{cc}
1 & 0\\
0 & 0
\end{array}\right), P^Z_1=\left(\begin{array}{cc}
0 & 0\\
0 & 1
\end{array}\right)$$

In the Ignis code, these matrices are defined in **tomography.fitters.utils.pauli_preparation_matrix**. We give an explicit definition here:

In [4]:
PX0 = 0.5*np.array([[1, 1], [1, 1]])
PX1 = 0.5*np.array([[1, -1], [-1, 1]])

PY0 = 0.5*np.array([[1, -1j], [1j, 1]])
PY1 = 0.5*np.array([[1, 1j], [-1j, 1]])

PZ0 = np.array([[1, 0], [0, 0]])
PZ1 = np.array([[0, 0], [0, 1]])

projectors = [PX0, PX1, PY0, PY1, PZ0, PZ1]

By Born's rule, $\text{tr}\left(P_{i}^{X}\rho\right)$ is the probability for the outcome $\left|i\right\rangle$ when measuring in the X-basis, and this probability can be estimated directly using repeated meausrements in the X-basis. The $Y$ and $Z$ bases are handled
similarily.

The computation $\text{tr}\left(P_{i}^{X}\rho\right)$ can be replaced by the scalar product $\vec{P}_i^X \cdot \vec{\rho}$ where $\vec{E}$ denotes the vector obtained from the operator $E$ by flattening its matrix (the result vector consists of the first row, then the second row etc.)

Now we can construct a matrix $$M=\left(\begin{array}{c}
\vec{P}_{0}^{X}\\
\vec{P}_{1}^{X}\\
\vec{P}_{0}^{Y}\\
\vec{P}_{1}^{Y}\\
\vec{P}_{0}^{Z}\\
\vec{P}_{1}^{Z}
\end{array}\right)$$

Such that $$M\vec{\rho}=\vec{p}=\left(\begin{array}{c}
p_{\left|0\right\rangle }^{X}\\
p_{\left|1\right\rangle }^{X}\\
p_{\left|0\right\rangle }^{Y}\\
p_{\left|1\right\rangle }^{Y}\\
p_{\left|0\right\rangle }^{Z}\\
p_{\left|1\right\rangle }^{Z}
\end{array}\right)$$

Is the equation relating the density operator to the observed probabilities.

In [5]:
M = np.array([p.flatten() for p in projectors])

Since $M$ can be computed by knowing the operators used in the tomography, and the vector $\vec{p}$ of probabilities can be estimated using the tomography results, all that remains is to solve the equation $M\vec{\rho}=\vec{p}$ for $\vec{\rho}$. If the rank of $M$ is large enough this can be done by multiplying both sides by $M^\dagger$:

$M^\dagger M\vec{\rho} = M^\dagger \vec{p}$

$\vec{\rho} = (M^\dagger M)^{-1} M^\dagger \vec{p}$

In our example, we obtain the matrix 
$$(M^\dagger M)^{-1} M^\dagger = \left(\begin{array}{cccccc}
\frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{4}{6} & -\frac{2}{6}\\
\frac{1}{2} & -\frac{1}{2} & \frac{i}{2} & -\frac{i}{2} & 0 & 0\\
\frac{1}{2} & -\frac{1}{2} & -\frac{i}{2} & \frac{i}{2} & 0 & 0\\
\frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & -\frac{2}{6} & \frac{4}{6}
\end{array}\right)$$

In [6]:
M_dg = np.conj(M).T
linear_inversion_matrix = np.linalg.inv(M_dg @ M) @ M_dg

Multiplication by the linear inversion matrix performs the reconstruction stage described earlier to obtain the density operator.

### Example: 2-qubit Linear inversion

For multiple qubit systems the technique of linear inversion remains the same. The projector operators are tensor products of 1-qubit projectors: $6^n$ projectors in total, since we measure according to $3^n$ operators (tensor products of $X,Y,Z$) and each operator has two projectors.

In [7]:
import itertools
projectors_2 = [np.kron(p1, p2) for (p1, p2) in itertools.product(projectors, repeat = 2)]
M_2 = np.array([p.flatten() for p in projectors_2])
M_dg_2 = np.conj(M_2).T
linear_inversion_matrix_2 = np.linalg.inv(M_dg_2 @ M_2) @ M_dg_2

We will now attempt to reconstruct the Bell state $\frac{\left|00\right\rangle +\left|11\right\rangle }{\sqrt{2}}$ from simulated tomography results. First, we prepare a quantum circuit which generates this bell state from the input $\left|00\right\rangle$.

In [8]:
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit

q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
bell.qasm()

'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q0[2];\nh q0[0];\ncx q0[0],q0[1];\n'

We now use Ignis' **state_tomography_circuits** procedure which generates the $3^n$ circuits obtained by adding to the bell circuit a measurement according to each of our measurement operators (Pauli by default). Then we execute on a standard simulator.

In [9]:
from qiskit import Aer
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo
qst_bell = tomo.state_tomography_circuits(bell, q2)
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)

Now we use the **tomography_data** procedure which converts the simulator output into the clear format used in tomography.

In [14]:
tomo_counts_bell = tomo.tomography_data(job.result(), qst_bell)
tomo_counts_bell

{('X', 'X'): {'00': 2435, '11': 2565},
 ('X', 'Y'): {'01': 1265, '00': 1233, '10': 1261, '11': 1241},
 ('X', 'Z'): {'01': 1285, '00': 1211, '10': 1288, '11': 1216},
 ('Y', 'X'): {'01': 1174, '00': 1290, '10': 1262, '11': 1274},
 ('Y', 'Y'): {'01': 2500, '10': 2500},
 ('Y', 'Z'): {'01': 1261, '00': 1268, '10': 1207, '11': 1264},
 ('Z', 'X'): {'01': 1243, '00': 1259, '10': 1264, '11': 1234},
 ('Z', 'Y'): {'01': 1247, '00': 1202, '10': 1256, '11': 1295},
 ('Z', 'Z'): {'00': 2531, '11': 2469}}

Now use **fitter_data** to explicitly extract the probability vector $\vec{p}$ and projector matrix $M$ that satisfy $M\vec{\rho} = \vec{p}$:

In [15]:
p, M = tomo.fitter_data(tomo_counts_bell)

Now we use the linear inversion technique to reconstructo $\vec{\rho}$. Since we usually represent density matrices as matrices and not vectors, we use Numpy's **reshape** function to convert $\vec{\rho}$ into $\rho$.

In [17]:
M_dg = np.conj(M).T
linear_inversion_matrix = np.linalg.inv(M_dg @ M) @ M_dg
rho_bell = linear_inversion_matrix @ p
rho_bell = np.reshape(rho_bell, (4, 4))
print(rho_bell)

[[ 5.01896365e-01+1.66190675e-18j -9.83005609e-03+4.69799865e-03j
  -3.25160364e-03-1.86632328e-03j  4.99957927e-01+3.85748848e-03j]
 [-9.83005609e-03-4.69799865e-03j  2.36980764e-06+1.12504089e-22j
  -2.16213462e-11-9.05093770e-03j -1.81660982e-03-1.04342867e-03j]
 [-3.25160364e-03+1.86632328e-03j -2.16213348e-11+9.05093770e-03j
  -2.40345122e-06+4.33911235e-22j  4.76476841e-03-1.79100465e-03j]
 [ 4.99957927e-01-3.85748848e-03j -1.81660982e-03+1.04342867e-03j
   4.76476841e-03+1.79100465e-03j  4.98019455e-01-3.05350746e-18j]]


To check the quality of our solution, we compute the fidelity between the real quantum state (obtained via simulation by a simulator that can return state vectors) and our calculated $\rho$. The closer the fidelity to 1, the better.

In [19]:
from qiskit.quantum_info import state_fidelity
job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity linear inversion =', F_bell)

Fit Fidelity = 0.9999158365116748


## Maximum Likelihood 

Linear inversion works perfectly on accurate data, but tomography data is never fully accurate. Two obvious obstacles are
1. Since the number of measurements is limited, we do not obtain the probability vector $\vec{p}$ but an approximation.
2. The measurement process might be noisy.

This may result in non-accurate or even self-contradicting $\vec{p}$, and the result of linear inversion might not be a density function at all (e.g. not nonnegative, or trace different than 1).

Since we want to solve the linear problem $A\vec{x}=\vec{p}$ for $x$, we can turn it into an optimization problem by attempting to minimize $\|A\vec{x}-\vec{p}\|_2$ while subjacting $x$ to additional constraints verifying this is indeed a density matrix. This is done by **state_cvx_fit**.

Another approach is to solve this optimization problem with no further constraints. The result might not be a density operator, i.e. positive semidefinite with trace 1; in this case the algorithm first rescales in order to obtain a density operator. This is done using **state_mle_fit**.

In [24]:
rho_cvx_bell = tomo.state_cvx_fit(p, M)
F_bell = state_fidelity(psi_bell, rho_cvx_bell)
print('Fit Fidelity CVX fit =', F_bell)

rho_mle_bell = tomo.state_cvx_fit(p, M)
F_bell = state_fidelity(psi_bell, rho_mle_bell)
print('Fit Fidelity MLE fit =', F_bell)

Fit Fidelity CVX fit= 1.006359274264286
Fit Fidelity MLE fit = 1.006359274264286
